In [ ]:
import os
import streamlit as st
import pickle
import time
from langchain.chains import RetrievalQA
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import UnstructuredURLLoader
from langchain.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings

# Set Gemini API Key
os.environ['GOOGLE_API_KEY'] = 'Enter your api key'

# Initialize LLM with Gemini
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0.9)

# Load data from URLs
loaders = UnstructuredURLLoader(urls=[
    "https://www.moneycontrol.com/news/business/markets/wall-street-rises-as-tesla-soars-on-ai-optimism-11351111.html",
    "https://www.moneycontrol.com/news/business/tata-motors-launches-punch-icng-price-starts-at-rs-7-1-lakh-11098751.html"
])
data = loaders.load()

# Split text into chunks (Reduced chunk size & overlap)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=100
)
docs = text_splitter.split_documents(data)

# Create embeddings using Gemini
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Create FAISS vector index
vector_index_path = "vector_index.faiss"

if os.path.exists(vector_index_path):
    try:
        vectorIndex = FAISS.load_local(vector_index_path, embeddings, allow_dangerous_deserialization=True)
    except Exception:
        vectorIndex = FAISS.from_documents(docs, embeddings)
        vectorIndex.save_local(vector_index_path)
else:
    vectorIndex = FAISS.from_documents(docs, embeddings)
    vectorIndex.save_local(vector_index_path)

# Define retrieval-based QA chain (Using "stuff" instead of MapReduce)
chain = RetrievalQA.from_chain_type(
    llm=llm, 
    retriever=vectorIndex.as_retriever(),
    chain_type="stuff"  # Efficient processing
)

# # Query Example
# query = "what is the price of Tiago iCNG?"

# import langchain
# langchain.debug = True

# response = chain({"query": query})
# print(response)
# Queries
queries = [
    "What is the price of Tiago iCNG?",
    "What are the main features of Punch iCNG?"
]

# Enable debugging
import langchain
langchain.debug = True

# Get responses for both queries
for query in queries:
    response = chain.invoke({"query": query})
    print(f"\nQuery: {query}\nResponse: {response}\n")

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "What is the price of Tiago iCNG?"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "What is the price of Tiago iCNG?",
  "context": "The Tiago iCNG is priced between Rs 6.55 lakh and Rs 8.1 lakh, while the Tigor iCNG comes at a price range of Rs 7.8 lakh to Rs 8.95 lakh.\n\nTata Motors Passenger Vehicles Ltd Head-Marketing, Vinay Pant said these introductions put together will make the company's CNG line up \"appealing, holistic, and stronger than ever\".\n\nPTI\n\nfirst published: Aug 4, 2023 02:17 pm\n\nGo PRO@₹1/dayPRO\n\nAdvertisement\n\nRemove Ad\n\nBusiness\n\nMarkets\n\nStocks\n\nEconomy\n\nCompanies\n\nTrends\n\nIPO\n\nOpinion\n\nEV Special\n\nOptions FestWebinar\n\nHomeNewsBusinessTata Motors launches Punch iCNG, price st

[llm/end] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain > llm:ChatGoogleGenerativeAI] [1.82s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "The Punch iCNG features twin-cylinder technology and enhanced safety features.  These safety features include a micro-switch that keeps the car off during refueling and thermal incident protection that cuts off CNG supply and releases gas into the atmosphere in case of a thermal incident.",
        "generation_info": {
          "finish_reason": "STOP",
          "safety_ratings": []
        },
        "type": "ChatGeneration",
        "message": {
          "lc": 1,
          "type": "constructor",
          "id": [
            "langchain",
            "schema",
            "messages",
            "AIMessage"
          ],
          "kwargs": {
            "content": "The Punch iCNG features twin-cylinder technology and enhanced safety features.  These safety features include a micro-switch that 